In [29]:
import torch
import os
import numpy as np
from torchvision import datasets, transforms, models


In [30]:
train_dir = os.path.join('dataset', 'part_one_dataset', 'train_data')
eval_dir = os.path.join('dataset', 'part_one_dataset', 'eval_data')
train_path = os.path.join(train_dir, '1_train_data.tar.pth')
eval_path = os.path.join(eval_dir, '1_eval_data.tar.pth')

t = torch.load(train_path, weights_only = False)

In [31]:
from torchvision import models
import torch

# Load a pre-trained ResNet model
resnet =  models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])  # Remove the last layer
resnet.eval()  # Set to evaluation mode

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = resnet.to(device)

In [32]:
features= t['data']

In [36]:
from sklearn.cluster import KMeans

# If features is already a NumPy array, skip the conversion
if isinstance(features, torch.Tensor):
    features_np = features.cpu().numpy() if features.is_cuda else features.numpy()
else:
    features_np = features  # Assume features is already a NumPy array
# Define number of prototypes (clusters)
num_prototypes = 5  # Adjust this based on the dataset
print("Original shape of features_np:", features_np.shape)
features_np_reshaped = features_np.reshape(features_np.shape[0], -1)
print("Reshaped features shape:", features_np_reshaped.shape)

Original shape of features_np: (2500, 32, 32, 3)
Reshaped features shape: (2500, 3072)


In [37]:
# Fit k-means with reshaped features
kmeans = KMeans(n_clusters=num_prototypes, random_state=42)
kmeans.fit(features_np_reshaped)
# The cluster centers are your initial prototypes
prototypes = kmeans.cluster_centers_

In [38]:
# Calculate distances of each point to its cluster center
distances = np.linalg.norm(features_np_reshaped - kmeans.cluster_centers_[kmeans.labels_], axis=1)

# Convert distances to weights (closer points have higher weights)
# Normalize weights by inverting distances and scaling to a max of 1
weights = 1 / (distances + 1e-8)  # Add a small constant to avoid division by zero
weights /= weights.max()  # Normalize to [0, 1]

# Output sample weights for inspection
weights[:10]  # Display the first 10 weights

array([0.41108547, 0.30464587, 0.35339062, 0.70255189, 0.340754  ,
       0.33898703, 0.38654109, 0.35468545, 0.30117097, 0.31768781])